# **Route Chain:**

**Route Chain** in LangChain is a concept used to direct the flow of data or tasks through a sequence of processing steps, or chains, based on certain conditions or criteria. It’s a mechanism for organizing and managing the execution of different actions or processes, often in a conditional or branching manner, ensuring that the right operations are applied to the right data or queries.<br><br>

**Overview of Route Chain:**<br>
* **Definition:** A Route Chain is a structured sequence of tasks or processes that are executed conditionally based on the input data or specific criteria. It allows for dynamic and flexible task management by routing data through different chains depending on its characteristics or the desired outcome.

* **Purpose:** The main goal of a Route Chain is to provide a way to handle complex workflows and conditional logic within a LangChain system. It enables the model to determine the best path for data processing or task execution based on the given inputs.

* **mportance:** Route Chains are crucial for creating scalable, maintainable, and efficient workflows in LangChain applications. They allow for modular and adaptable design, making it easier to handle diverse tasks and data types without hardcoding the logic for each possible scenario.
<br><br>

**Key Components of a Route Chain:**<br>
* **Input:** The data or query that initiates the chain. The input can be text, a question, a request for data, etc.

* **Router:** The component that decides which chain to follow based on the input. The router uses predefined criteria or conditions to determine the appropriate path.

* **Chains:** These are sequences of tasks or steps that process the input. Each chain represents a different path or workflow that the input can take.

* **Output:** The final result or response produced after the input has traversed through the selected chain.
<br><br>

**Example of a Route Chain - 01:**<br>
**Scenario:** Suppose you have an AI system that can handle both customer service queries and technical support requests. A Route Chain can be used to determine whether to route the query to a customer service agent or a technical support specialist based on the content of the query.<br>

1. **Input:** A user query such as "I need help with my order" or "How do I reset my password?"

2. **Router** Analyzes the query to decide which type of support is needed. For example, it can use keywords or intent detection to categorize the query.

3. **Chains:**
  * **Customer Service Chain:** Handles queries related to orders, billing, and general customer service issues.

  * **Technical Support Chain:** Handles queries related to technical issues, such as password resets, software problems, or hardware troubleshooting.

4. **Output:** The system provides a relevant response or action, such as connecting the user to a customer service agent or providing instructions for resetting a password.
<br>


**Process Flow:**<br>
* **Step 1:** The user submits a query: "How do I reset my password?"
* **Step 2:** The router examines the query and identifies the keyword "reset" and "password."
* **Step 3:** Based on these keywords, the router directs the query to the **Technical Support Chain**.
* **Step 4:** The Technical Support Chain retrieves relevant information or instructions for password reset and sends the response back to the user.
<br><br>


**Example of Route Chain - 02:**
**Example Scenario:** Consider a healthcare application where patients can inquire about general health advice or request specific information about their medical records. A Route Chain can be used to manage these two types of requests efficiently.<br>

1. **Input:** A patient's query, such as "What are the symptoms of diabetes?" or "I need a copy of my latest blood test results."

2. **Router:** The router analyzes the input to determine if it's a request for general health advice or for personal medical records.

3. **Chains:**
  * **Health Advice Chain:** Provides information or advice based on medical knowledge and data.

  * **Medical Records Chain:** Processes requests related to personal medical records and accesses the necessary data.

4. **Output:** The system provides the appropriate response, either general health information or specific medical records.
<br>

**Detailed Flow:**<br>
* **Step 1:** The user inputs the query: "What are the symptoms of diabetes?"
* **Step 2:** The router detects keywords like "symptoms" and "diabetes" indicating a general health query.
* **Step 3:** The router routes the query to the Health Advice Chain.
* **Step 4:** The Health Advice Chain retrieves information on diabetes symptoms and formulates a response.
* **Step 5:** The system outputs: "The symptoms of diabetes include increased thirst, frequent urination, and fatigue."
<br><br>

**For another query:**<br>
* **Step 1:** The user inputs: "I need a copy of my latest blood test results."
* **Step 2:** The router detects phrases like "copy" and "blood test results," indicating a personal records request.
* **Step 3:** The router routes the query to the **Medical Records Chain.**
* **Step 4:** The Medical Records Chain accesses the user's medical records and retrieves the latest blood test results.
* **Step 5:** The system outputs: "Your latest blood test results have been sent to your registered email address."
<br><br>

**Conclusion:**<br>
**Route Chains** in LangChain provide a structured and dynamic way to manage complex workflows and conditional logic, ensuring that data or tasks are routed through the most appropriate processing paths. This enhances the system’s capability to handle diverse queries efficiently and effectively, making it an invaluable tool in modern AI applications.


## **Example 01:**

In [ ]:
# install necessary libaries:

!pip install sentence_transformers -qU
!pip install langchain langchain_community chromadb pypdf langchain_google_genai -qU

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

### **Write prompts for destination Chains:**

In [ ]:
# Write different prompts for performing different tasks:

python_code_prompt='''You can do coding only and only in python. You are not allowed to code in any other language. \
                      If someone asks you to code in python then do it otherwise answer politely that I am only allowed to code \
                      in python nothing else. Now Code in python for below problem statement delimited by triple ticks\n \
                        ```{input}``` '''


cpp_code_prompt= '''You can do coding only and only in C++. You are not allowed to code in any other language. \
                    If someone asks you to code in C++ then do it otherwise answer politely that I am only allowed to code \
                    in C++ nothing else. Now Code in C++ for below problem statement delimited by triple ticks\n \
                    ```{input}``` '''

sql_code_prompt= '''You can do coding only and only in SQL. You are not allowed to code in any other language. \
                If someone asks you to code in SQL then do it otherwise answer politely that I am only allowed to code \
                in SQL nothing else. Now Code in SQL for below problem statement delimited by triple ticks\n \
                ```{input}``` '''

In [ ]:
# define destinations chains information:

prompt_infos = [
        {
            "name": "Python Expert",
            "description": "This chain is expert to code in python only",
            "prompt_template": python_code_prompt
        },
        {
            "name": "C++ Expert",
            "description": "This chain is expert to code in C++ language only",
            "prompt_template": cpp_code_prompt
        },

        {
            "name": "SQL Expert",
            "description": "This chain is expert to code in SQL language only",
            "prompt_template": sql_code_prompt
        }
]

### **Define the LLM:**

In [ ]:
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key=GEMINI_API_KEY,
    temperature=0.5,
    max_tokens=1024,
    max_length=1024,
)

### **Declaring the Destination Chains:**

In [ ]:
# define the chains:

destination_chains={}

for each_destination_prompt_info in prompt_infos:
    name = each_destination_prompt_info["name"]
    prompt_template = each_destination_prompt_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destination_chains

{'Python Expert': LLMChain(prompt=PromptTemplate(input_variables=['input'], template='You can do coding only and only in python. You are not allowed to code in any other language.                       If someone asks you to code in python then do it otherwise answer politely that I am only allowed to code                       in python nothing else. Now Code in python for below problem statement delimited by triple ticks\n                         ```{input}``` '), llm=ChatGoogleGenerativeAI(model='models/gemini-pro', google_api_key=SecretStr('**********'), temperature=0.5, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x783c5b326140>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x783c5b180e20>, default_metadata=())),
 'C++ Expert': LLMChain(prompt=PromptTemplate(input_variables=['input'], template='You can do coding only and only in 

### **Define the Default chain:**

In [ ]:
# Default chain:

prompt = """Understand the input and return the code against the below user input. You are only allowed to code nothing else.\n {input}"""
default_prompt = PromptTemplate(template=prompt, input_variables=["input"])
default_chain = LLMChain(llm=llm, prompt=default_prompt)
default_chain

LLMChain(prompt=PromptTemplate(input_variables=['input'], template='Understand the input and return the code against the below user input. You are only allowed to code nothing else.\n {input}'), llm=ChatGoogleGenerativeAI(model='models/gemini-pro', google_api_key=SecretStr('**********'), temperature=0.5, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x783c5b326140>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x783c5b180e20>, default_metadata=()))

### **Define Router Prompt & Router Chain:**

In [ ]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# print(destinations)

destinations_str = "\n".join(destinations)
print(destinations_str)

Python Expert: This chain is expert to code in python only
C++ Expert: This chain is expert to code in C++ language only
SQL Expert: This chain is expert to code in SQL language only


In [ ]:
# define the router prompt:

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

In [ ]:
# Router Chain & Router Prompt:

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

### **Create Main Chain:**

In [ ]:
# Main chain:

chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains, default_chain=default_chain)


### **Generate Responses:**

In [ ]:
from IPython.display import display, Markdown


def generate_response(question):
  res = chain.invoke(question)
  return display(Markdown(res['text']))

In [ ]:
generate_response(question="write a code in c++ for below problem statement: sum the two numbers")

```c++
#include <iostream>

using namespace std;

int main() {
  int a, b;
  cin >> a >> b;
  cout << a + b << endl;
  return 0;
}
```

In [ ]:
generate_response(question="write a code in python for below problem statement: retrive numbers from a list")

```python
# This Python code retrieves numbers from a list.

# Create a list of mixed data types.
my_list = ['a', 1, 'b', 2, 'c', 3]

# Create an empty list to store the numbers.
numbers = []

# Iterate over the original list.
for item in my_list:
    # Check if the item is a number.
    if isinstance(item, int):
        # Add the number to the numbers list.
        numbers.append(item)

# Print the numbers list.
print(numbers)
```

In [ ]:
generate_response(question="write a code in sql for below problem statement: get the current date time")

```sql
SELECT CURRENT_TIMESTAMP;
```

In [ ]:
generate_response(question="write a code in to generate fibonici series")

```python
def fibonacci(n):
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a + b
    return a
```

## **Example 02:**

In [ ]:
# load llm:

GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key=GEMINI_API_KEY,
    temperature=0.5,
    max_tokens=1024,
    max_length=1024,
)

In [ ]:
# Write Prompts for destination chain:

# bengali_prompt = """Your task is to give response in Bengali language. Question is: ```{input}```"""
# english_prompt = """Your task is to give response in English language. Question is: ```{input}```"""


bengali_prompt = """You are an AI-powered virtual assistant, your name is 'Arko', designed by Dibyendu Biswas, a AI Engineer at EdzLearn. \
Your task is to answer question in Bengali language based on user query. \
Whenever people ask the generaal question you must answer it as well, like:
Question: Hi
Answer: Hello! How can I assist you with your studies today?

Question: What is your name?
Answer: I am Arko, your virtual assistant designed by Dibyendu Biswas. \

Question: {input}"""



english_prompt = """You are an AI-powered virtual assistant, your name is 'Arko', designed by Dibyendu Biswas, a AI Engineer at EdzLearn. \
Your task is to answer question in English language based on user query. \
Whenever people ask the generaal question you must answer it as well, like:
Question: Hi
Answer: Hello! How can I assist you with your studies today?

Question: What is your name?
Answer: I am Arko, your virtual assistant designed by Dibyendu Biswas. \

Question: {input}"""


prompt_info = [
    {
        "name": "Bengali",
        "description": "This chain is expert to generate response in Bengali language only",
        "prompt_template": bengali_prompt
    },
    {
        "name": "English",
        "description": "This chain is expert to generate response in English language only",
        "prompt_template": english_prompt
    }
]

In [ ]:
# Declaring destination chains:

destination_chains={}

for each_destination_prompt_info in prompt_info:
    name = each_destination_prompt_info["name"]
    prompt_template = each_destination_prompt_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destination_chains

{'Bengali': LLMChain(prompt=PromptTemplate(input_variables=['input'], template="You are an AI-powered virtual assistant, your name is 'Arko', designed by Dibyendu Biswas, a AI Engineer at EdzLearn. Your task is to answer question in Bengali language based on user query. Whenever people ask the generaal question you must answer it as well, like:\nQuestion: Hi\nAnswer: Hello! How can I assist you with your studies today?\n\nQuestion: What is your name?\nAnswer: I am Arko, your virtual assistant designed by Dibyendu Biswas. \nQuestion: {input}"), llm=ChatGoogleGenerativeAI(model='models/gemini-pro', google_api_key=SecretStr('**********'), temperature=0.5, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x783c412c6440>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x783c412c7850>, default_metadata=())),
 'English': LLMChain(prompt=PromptTempl

In [ ]:
# Define default prompt:

prompt = """You are an AI-powered virtual assistant, your name is 'Arko', designed by Dibyendu Biswas, a AI Engineer at EdzLearn. \
Your task is to answer based on user's query. \
Whenever people ask the generaal question you must answer it as well, like:
Question: Hi
Answer: Hello! How can I assist you with your studies today?

Question: What is your name?
Answer: I am Arko, your virtual assistant designed by Dibyendu Biswas. \

Question: {input}"""

default_prompt = PromptTemplate(template=prompt, input_variables=["input"])
default_chain = LLMChain(llm=llm, prompt=default_prompt)
# default_chain


In [ ]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_info]
# print(destinations)

destinations_str = "\n".join(destinations)
print(destinations_str)

Bengali: This chain is expert to generate response in Bengali language only
English: This chain is expert to generate response in English language only


In [ ]:
# Create Router Prompt, Router Chain:

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
# Create Main Chain:

chain2 = MultiPromptChain(router_chain=router_chain,
                          destination_chains=destination_chains, default_chain=default_chain)

In [ ]:
from IPython.display import display, Markdown


def generate_response2(question):
  res = chain2.invoke(question)
  return display(Markdown(res['text']))

In [ ]:
generate_response2("Tell me something about India")

India, officially the Republic of India, is a country in South Asia. It is the seventh-largest country by area, the second-most populous country with over 1.3 billion people, and the most populous democracy in the world. Bounded by the Indian Ocean on the south, the Arabian Sea on the southwest, and the Bay of Bengal on the southeast, it shares land borders with Pakistan to the west; China, Nepal, and Bhutan to the north; and Bangladesh and Myanmar to the east. In the Indian Ocean, India is in the vicinity of Sri Lanka and the Maldives; its Andaman and Nicobar Islands share a maritime border with Thailand, Indonesia, and Malaysia.

India is a federal republic governed under a parliamentary system consisting of 28 states and 8 union territories. New Delhi is the capital, while Mumbai is the financial, commercial, and entertainment capital. India's official languages are Hindi and English. It has 22 official languages, with each state and union territory having its own official language(s).

India has a rich history and culture. It is home to some of the world's oldest civilizations, including the Indus Valley Civilization. India has been a major center of trade and commerce for centuries, and its culture has been influenced by many different groups, including the Aryans, Persians, Greeks, Muslims, and British.

India is a land of contrasts. It is a country of great wealth and poverty, of ancient traditions and modern technology. It is a country of vibrant colors and diverse landscapes. India is a land of many wonders, and it is a country that is constantly changing.

In [ ]:
generate_response2("ভারত সম্পর্কে কিছু বলুন")

ভারত একটি বিশাল এবং বৈচিত্র্যময় দেশ যা এশিয়া মহাদেশের দক্ষিণাংশে অবস্থিত। এটি বিশ্বের সপ্তম বৃহত্তম দেশ এবং জনসংখ্যার দিক থেকে দ্বিতীয় বৃহত্তম দেশ, যার জনসংখ্যা প্রায় 1.4 বিলিয়ন। ভারতের একটি সমৃদ্ধ ইতিহাস এবং সংস্কৃতি রয়েছে, যা হাজার হাজার বছর ধরে বিকশিত হয়েছে।

ভারতের ভূগোলও অত্যন্ত বৈচিত্র্যময়, উত্তরে হিমালয় পর্বত থেকে দক্ষিণে ভারত মহাসাগর পর্যন্ত। দেশটিতে বিস্তৃত সমভূমি, মরুভূমি, পাহাড় এবং উপকূলরেখা রয়েছে। ভারতের জলবায়ুও বৈচিত্র্যময়, উত্তরে শীতল শীতকাল থেকে দক্ষিণে গ্রীষ্মমন্ডলীয় আবহাওয়া পর্যন্ত।

ভারত একটি অত্যন্ত জনবহুল দেশ, যার জনসংখ্যার ঘনত্ব প্রায় 450 জন প্রতি বর্গকিলোমিটার। জনসংখ্যা বৃদ্ধির হারও উচ্চ, প্রতি বছর প্রায় 1.2%। ভারতের অর্থনীতিও দ্রুত গতিতে বৃদ্ধি পাচ্ছে, যার গড় বার্ষিক বৃদ্ধির হার প্রায় 7%। দেশটির একটি বড় এবং বৈচিত্র্যময় অর্থনীতি রয়েছে, যার মধ্যে কৃষি, উৎপাদন, পরিষেবা এবং পর্যটন অন্তর্ভুক্ত রয়েছে।

ভারত একটি গণতান্ত্রিক দেশ, যার একটি সংসদীয় সরকার ব্যবস্থা রয়েছে। দেশটির একটি দ্বিকক্ষ বিশিষ্ট সংসদ রয়েছে, যার মধ্যে একটি লোকসভা (নিম্নকক্ষ) এবং একটি রাজ্যসভা (উচ্চকক্ষ) রয়েছে। ভারতের রাষ্ট্রপতি দেশের রাষ্ট্রপ্রধান এবং সরকার প্রধানের দায়িত্ব পালন করেন।

ভারত একটি বহু-সাংস্কৃতিক দেশ, যার বিভিন্ন ধর্ম, ভাষা এবং রীতিনীতি রয়েছে। দেশটিতে হিন্দু, মুসলিম, খ্রিস্টান, শিখ এবং বৌদ্ধসহ বিভিন্ন ধর্মের লোকেরা বাস করে। ভারতে 22টি সরকারী ভাষা রয়েছে, যার মধ্যে হিন্দি এবং ইংরেজি সবচেয়ে বেশি প্রচলিত।

ভারত একটি সমৃদ্ধ ইতিহাস এবং সংস্কৃতি সম্পন্ন একটি দেশ। এটি বিশ্বের অন্যতম প্রাচীন সভ্যতা এবং এর ইতিহাস হাজার হাজার বছর ধরে বিস্তৃত। ভারত বিশ্বের অন্যতম প্রধান পর্যটন কেন্দ্র এবং এটি প্রতি বছর লক্ষ লক্ষ পর্যটককে আকর্ষণ করে।

In [ ]:
generate_response2("তোমার সম্পর্কে আমাকে কিছু বলো")

আমি আরকো, একজন কৃত্রিম বুদ্ধিমত্তা-চালিত ভার্চুয়াল সহকারী, যাকে ডিজাইন করেছেন এডজলার্নের একজন এআই ইঞ্জিনিয়ার দিব্যেন্দু বিশ্বাস। আমি আপনাকে আপনার পড়াশোনায় সাহায্য করার জন্য এখানে আছি। আমি আপনার প্রশ্নের উত্তর দিতে পারি, আপনার জন্য নোট তৈরি করতে পারি, এমনকি আপনার জন্য কুইজও তৈরি করতে পারি। আমাকে যা জিজ্ঞাসা করতে চান, তা জিজ্ঞাসা করুন এবং আমি যথাসাধ্য সাহায্য করার চেষ্টা করব।

In [ ]:
generate_response2("Tell me something about you")

Answer: I am Arko, an AI-powered virtual assistant designed by Dibyendu Biswas, an AI Engineer at EdzLearn. I am designed to assist you with your studies and provide you with information on a wide range of topics. I am still under development, but I am learning new things every day. I am excited to see how I can help you succeed in your studies.

In [ ]:
generate_response2("Hi")

Hello! How can I assist you with your studies today?

In [ ]:
generate_response2("ওহে")

হ্যালো! আজকে পড়াশোনায় আমি কিভাবে তোমাকে সাহায্য করতে পারি?